In [83]:
import pandas as pd
import  math
import os
import bson
from dateutil import parser

from pymongo import MongoClient

def mongoCollection(connstr, db, collection):
    client = MongoClient(connstr)
    db = client[db]
    return db[collection]

In [59]:
collection = mongoCollection(os.environ.get('MONGODB_CONN_STR'), 'smsinfo', 'transactions')

In [12]:
csvdf = pd.read_csv('/data/2020-March.csv') 

In [14]:
csvdf.to_dict()['_id'][0]

'5e80d16248a45dfa05b4e711'

In [90]:
keys = list(csvdf.to_dict().keys())
count = len(csvdf)

In [31]:
def dict_from_idx(dataframe, idx, keys):
    data = {}
    for k in keys:
        data[k] = dataframe[k][idx]
    return data

In [93]:
for i in range(count):
    d = dict_from_idx(csvdf, i, keys)
    date = d.pop('transaction.datetime')
    _id = d.pop('_id')
    if type(_id) != str and math.isnan(float(_id)):
        d.update({'transaction.datetime':  parser.parse(date)})
        print(d)
        for key in list(d.keys()):
            _keys = key.split('.')
            if len(_keys) == 2:
                val = d.pop(key)
                d.update({_keys[0]: {_keys[1]: val}})
                print(d)
        print(d)        
        _id = collection.insert_one(d).inserted_id
        print(d)
#     else:
#         collection.update_one({'_id': bson.ObjectId(_id)}, {'$set': d}, upsert = False)
        print(_id)
#     print(d)
        

{'transaction.expense_amount': 2000.0, 'transaction.payment_mode': 'Cash', 'transaction.merchant': 'Drdo', 'transaction.category': 'grocery', 'transaction.sub_category': 'all', 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'event.name': nan, 'transaction.comment': nan, 'transaction.datetime': datetime.datetime(2020, 3, 29, 18, 53)}
{'transaction.payment_mode': 'Cash', 'transaction.merchant': 'Drdo', 'transaction.category': 'grocery', 'transaction.sub_category': 'all', 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'event.name': nan, 'transaction.comment': nan, 'transaction.datetime': datetime.datetime(2020, 3, 29, 18, 53), 'transaction': {'expense_amount': 2000.0}}
{'transaction.merchant': 'Drdo', 'transaction.category': 'grocery', 'transaction.sub_category': 'all', 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'event.name': nan, 'transaction.comment': nan, 'transaction.datetime': datetime.datetime(2020, 3, 29, 1

{'transaction': {'datetime': datetime.datetime(2020, 3, 20, 20, 38, 6)}, 'event': {'name': nan}, '_id': ObjectId('5e84a2f2e0428e02c2b4e98c')}
5e84a2f2e0428e02c2b4e98c


In [44]:
d = {'a': 1, 'b': 2}
d.pop('a')

1

In [61]:
d.update({'a': 2})

In [62]:
d

{'transaction.expense_amount': 500.0,
 'transaction.payment_mode': 'CREDIT Card xx3690',
 'transaction.merchant': 'Akshayakalpa',
 'transaction.category': 'None',
 'transaction.sub_category': 'None',
 'transaction.direction': 'TRANSFER',
 'transaction.service_provider': nan,
 'event.name': nan,
 'transaction.comment': nan,
 'a': 2}

In [64]:
d0 = dict_from_idx(csvdf, 0, keys)
date0 = d0.pop('transaction.datetime')
_id0 = d0.pop('_id')

In [69]:
d0.update({'a': 1})
d0

{'transaction.expense_amount': 240.0,
 'transaction.payment_mode': 'Akshayakalpa Wallet',
 'transaction.merchant': 'Akshayakalpa',
 'transaction.category': 'grocery',
 'transaction.sub_category': 'diary',
 'transaction.direction': 'DEBIT',
 'transaction.service_provider': nan,
 'event.name': nan,
 'transaction.comment': nan,
 'a': 1}

In [78]:
c = collection.update_one({'_id': bson.ObjectId(_id0)}, {'$set': d0}, upsert = False)

In [79]:
_id0

'5e80d16248a45dfa05b4e711'

In [80]:
c.matched_count

1

In [81]:
d0

{'transaction.expense_amount': 240.0,
 'transaction.payment_mode': 'Akshayakalpa Wallet',
 'transaction.merchant': 'Akshayakalpa',
 'transaction.category': 'grocery',
 'transaction.sub_category': 'diary',
 'transaction.direction': 'DEBIT',
 'transaction.service_provider': nan,
 'event.name': nan,
 'transaction.comment': nan,
 'a': 1}

In [ ]:
from dateutil import parser